# import a csv file made in QGIS, and convert to Google Maps-enabled coords for wiki pages

In [2]:
import pandas

In [3]:
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/australia/australia-route-fixing-bassgas.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/australia/australia-route-fixing.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/australia/australia-route-fixing-darling-downs.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/report-gas-pipelines-2022/figures-and-mapping/from-scott/mountaineer-pipeline.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/pipelines/mapping/2022-oil/tazama/tazama-pipeline.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/pipelines/mapping/2022-oil/report-mapping/willow-pipeline.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/pipelines/mapping/fixing-vts/new-vts.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/pipelines/mapping/2022-gas/nigeria-morocco/nigeria-morocco-wags-elps.csv')
coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/pipelines/mapping/2022-gas/hydrogen/fluxys-hydrogen-connector.csv')

* look at the csv file to understand how it's organized
* ultimately each x,y pair should look like:  x1,y1 : x2,y2 : x3,y3 : ...

In [4]:
coords_csv

,id,name,vertex_index,vertex_part,vertex_part_index,distance,angle,lon,lat
0,NaN,hydrogen-fluxys,0,0,0,0.000000,8.457181,3.760226,51.077952
1,NaN,hydrogen-fluxys,1,0,1,0.043330,33.850563,3.766599,51.120810
2,NaN,hydrogen-fluxys,2,0,2,0.167978,68.882078,3.873715,51.184554
3,NaN,hydrogen-fluxys,3,0,3,0.371170,83.340380,4.072842,51.224993
4,NaN,hydrogen-fluxys,4,0,4,0.424756,85.273533,4.126401,51.226713
5,NaN,hydrogen-fluxys,5,0,5,0.548066,87.521787,4.248624,51.243051
6,NaN,hydrogen-fluxys,6,0,6,0.802266,96.294585,4.502551,51.231266
7,NaN,hydrogen-fluxys,7,0,7,1.123269,102.993455,4.818743,51.175900
8,NaN,hydrogen-fluxys,8,0,8,1.259375,105.494313,4.949541,51.138258
9,NaN,hydrogen-fluxys,9,0,9,1.324355,103.800463,5.012326,51.121513


* create a list of pipeline names that match up exactly with the csv file; this is something you'll edit yourself

* _aside: to pull out all the rows where the Name column matches "Example Pipeline 1", use the syntax below:_

* now loop through each ```pipeline_name``` in this list and do the appropriate reformatting, and store that in ```list_of_formatted_coords```

### if from the EIA dataset, the pipelines don't have individual names

in QGIS, select the given pipeline(s), then Vector > Collect geometries, then add $x $y (lon, lat) and export as a csv file; then you'll use the "vertex_part" label as the pipeline name

### if normal separate segments

In [5]:
segment_name = 'name'
#segment_name = 'PipelineName'
#segment_name = 'LINE_NO'
#segment_name = 'OBJECTID'

In [6]:
#list_of_pipelines = ["Tioga to Emerson Pipeline"]
list_of_pipelines = list(sorted(set(coords_csv[segment_name])))
#list_of_pipelines = list(set(coords_csv['PipelineName']))
#print(list_of_pipelines)
print(sorted(list(list_of_pipelines)))

['hydrogen-fluxys']


In [7]:
list_of_formatted_coords = []
for pipeline_name in list_of_pipelines:
    # pull out all rows matching pipeline_name
#    pipeline_specific_dataframe = coords_csv.loc[coords_csv['PipelineName']==pipeline_name]
    pipeline_specific_dataframe = coords_csv.loc[coords_csv[segment_name]==pipeline_name]
    # then change y,x coords into strings
    pipeline_specific_dataframe[['lat','lon']].astype(str)
    # join those strings with a comma
    joined_strings_dataframe = pipeline_specific_dataframe[['lat', 'lon']].astype(str).apply(lambda x: ','.join(x), axis=1)
    # add the colon
    joined_strings_with_colon = joined_strings_dataframe.str.cat(sep=':')
    # store this in a list
    list_of_formatted_coords.append(joined_strings_with_colon)

In [8]:
' ; '.join(list_of_formatted_coords).__len__()

280

In [9]:
for index,pipeline_name in enumerate(list_of_pipelines):
    print(';'); print(list_of_formatted_coords[index])

;
51.077952,3.760226:51.12081,3.766599:51.184554,3.873715:51.224993,4.072842:51.226713,4.126401:51.243051,4.248624:51.231266,4.502551:51.1759,4.818743:51.138258,4.949541:51.121513,5.012326:51.107225,5.075896:51.098355,5.137111:51.05891,5.214808:50.751235,5.666823:50.744336,5.724149


### if it's a big network AND segments are not separated, you gotta deal with the "vertex_part" info

In [22]:
#list_of_pipelines = ['elps1', 'elps2']
list_of_pipelines = ['nm1', 'nm2', 'nm3', 'nm4', 'nm5', 'nm6', 'nm7', 'nm8', 'nm9', 'nm10', 'nm11']
#list_of_pipelines = ['wag1', 'wag2', 'wag3', 'wag4']




In [23]:
list_of_formatted_coords = []

for pipeline_name in list_of_pipelines: 
    list_of_vertex_parts = list(set(coords_csv[coords_csv[segment_name]==pipeline_name]['vertex_part']))
    
    for vertex_part in list_of_vertex_parts:
        # pull out all rows matching pipeline_name
        pipeline_specific_dataframe = coords_csv.loc[(coords_csv[segment_name]==pipeline_name)&(coords_csv['vertex_part']==vertex_part)]
        # then change y,x coords into strings
        pipeline_specific_dataframe[['lat','lon']].astype(str)
        # join those strings with a comma
        joined_strings_dataframe = pipeline_specific_dataframe[['lat', 'lon']].astype(str).apply(lambda x: ', '.join(x), axis=1)
        # add the colon
        joined_strings_with_colon = joined_strings_dataframe.str.cat(sep=':')
        # store this in a list
        list_of_formatted_coords.append(joined_strings_with_colon)

* finally, write some code to spit out the information in the format you want

In [24]:
len(list_of_formatted_coords)

11

In [25]:
for i in list_of_formatted_coords:
    print(i)
    print(';')

4.135867, -1.449023:4.014729, -1.911063:3.892177, -2.254852:3.817065, -2.549708:3.773246, -2.976308:3.769251, -3.463418:3.877895, -3.967016:3.959369, -4.457004:4.000103, -4.715609:3.986525, -5.123932:3.972947, -5.505034:3.932212, -5.668363:3.823575, -6.049465:3.728506, -6.376123:3.660594, -6.702782:3.592676, -7.070273:3.565508, -7.505818:3.565508, -7.954973:3.565508, -8.376907:3.647011, -8.812452:3.809994, -9.179943:4.176592, -9.901314:4.407326, -10.350469:4.624423, -10.677128:4.760074, -10.922122:5.261746, -11.684325:5.586942, -12.133481:5.911957, -12.569025:6.196188, -12.909295:6.53436, -13.317618:6.845276, -13.603444:7.196502, -13.916492:7.426003, -14.107043:7.763287, -14.365647:8.086826, -14.515366:8.423569, -14.678695:8.827273, -14.882857:9.270837, -15.087018:9.83456, -15.345623:9.968641, -15.413677:10.129464, -15.631449:10.31699, -15.849222:10.544549, -16.121437:10.825419, -16.393653:11.065956, -16.625036:11.252904, -16.856419:11.45307, -17.006138:11.852976, -17.360018:12.079332,